### Create worm datasets of various sizes by subsampling a larger combined datasets
---
*Last updated: 14 November 2023*

In [9]:
import os
import pickle

from pprint import PrettyPrinter
from itertools import combinations
from utils import init_random_seeds
from data._utils import generate_all_subsets, generate_subsets_of_size

In [15]:
# Set random seed for reproducibility
seed = 42
init_random_seeds(seed)

# Settings for pretty printing
pp = PrettyPrinter(indent=4, width=100, compact=True)

# Load combined dataset and print some information about it
datadir = "combined_dataset"
file = os.path.join(datadir, "combined_dataset.pickle")
combined_dataset = pickle.load(open(file, "rb"))  # a few GBs large
num_worms = len(combined_dataset.keys())
print(f"number of worms (N): {num_worms}", end="\n\n")
print(f"all worm Ids:\n {list(combined_dataset.keys())}", end="\n\n")
print(f"data keys for each worm' data:")
pp.pprint(list(combined_dataset["worm0"].keys()))
print()

# Example of choosing all subsets of size N-1 of the wormIDs when N
all_worm_ids = list(combined_dataset.keys())
worm_subsets = combinations(all_worm_ids, len(all_worm_ids) - 1)
for i, subset in enumerate(worm_subsets):
    print(f"{i}: {subset[-5:]}")
    if i == 2:
        break
print()

# Generate 10 of all the possible size-3 subsets
# returns list of datasets containing 3 worms
subsets_3 = generate_subsets_of_size(combined_dataset, subset_size=3, max_subsets=10)
print(f"\ttype(subsets_3): {type(subsets_3)}")
print(f"\tlen(subsets_3): {len(subsets_3)}")
print(f"\ttype(subsets_3[0]): {type(subsets_3[0])}")
print()
for _, subset in enumerate(subsets_3):
    print(f"{_}: {tuple(subset.keys())}")
print()

# Generate 10 of each size-n subset for n from 1 to 20;
# returns dict mapping n to list of datasets containing n worms
all_subsets = generate_all_subsets(
    combined_dataset,
    max_subsets_per_size=10,
    max_size=20,
)
print(f"\ttype(all_subsets): {type(all_subsets)}")
print(f"\tlen(all_subsets): {len(all_subsets)}")
print()
for size, subsets in all_subsets.items():
    print(f"size {size}: \t num. subsets: {len(subsets)}")
print()

number of worms (N): 284

all worm Ids:
 ['worm0', 'worm1', 'worm2', 'worm3', 'worm4', 'worm5', 'worm6', 'worm7', 'worm8', 'worm9', 'worm10', 'worm11', 'worm12', 'worm13', 'worm14', 'worm15', 'worm16', 'worm17', 'worm18', 'worm19', 'worm20', 'worm21', 'worm22', 'worm23', 'worm24', 'worm25', 'worm26', 'worm27', 'worm28', 'worm29', 'worm30', 'worm31', 'worm32', 'worm33', 'worm34', 'worm35', 'worm36', 'worm37', 'worm38', 'worm39', 'worm40', 'worm41', 'worm42', 'worm43', 'worm44', 'worm45', 'worm46', 'worm47', 'worm48', 'worm49', 'worm50', 'worm51', 'worm52', 'worm53', 'worm54', 'worm55', 'worm56', 'worm57', 'worm58', 'worm59', 'worm60', 'worm61', 'worm62', 'worm63', 'worm64', 'worm65', 'worm66', 'worm67', 'worm68', 'worm69', 'worm70', 'worm71', 'worm72', 'worm73', 'worm74', 'worm75', 'worm76', 'worm77', 'worm78', 'worm79', 'worm80', 'worm81', 'worm82', 'worm83', 'worm84', 'worm85', 'worm86', 'worm87', 'worm88', 'worm89', 'worm90', 'worm91', 'worm92', 'worm93', 'worm94', 'worm95', 'worm96'